In [1]:
from pathlib import Path
import os
import pandas as pd
import pickle

In [2]:
num_dtb = 1
clean_data_folder = Path('../../clean_data')
dataset_dir = Path("../../data_for_model/")
version_name = f'dataset_{num_dtb}-output_khoa-string'

In [3]:
df = pd.read_parquet(clean_data_folder/'data_with_khoa.parquet')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100804 entries, 0 to 100803
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype   
---  ------   --------------   -----   
 0   mssv     100804 non-null  category
 1   hoc_ky   100804 non-null  category
 2   nam_hoc  100804 non-null  category
 3   dtb_hk   100804 non-null  float32 
 4   khoa     100804 non-null  category
dtypes: category(4), float32(1)
memory usage: 1.5 MB


In [5]:
new_dataset = {
    'input': [],
    'target': [],
    'mo_ta': 
f'''
==================     VERSION {version_name}     ==================
1. một mẩu của target là MỘTMỘTMỘT dtb_hk
2. một mẩu của input là list dtb_hk trong các học kỳ trước hk trong target
và điểm này thuộc sinh viên tương ứng target
3. diem được normalize từ [0-10] về [0-1]
4. input và target được sắp xếp theo thứ tự học kỳ tăng dần
5. input và target chưa được shuffle
6. Các học kỳ có giá trị null đã được loại bỏ. 
Xem như học kỳ trước và sau của học kỳ null đó là 2 học kỳ liên tiếp
7. HỌC KỲ 3 KHÔNGKHÔNGKHÔNG ĐƯỢC DROP
---
8. Input thêm khoa dạng string vào đầu tiên
    - Ví dụ input data_for_model v1, v2 là (7, 9)
    - Nếu sinh viên thuộc KHMT, input của dataset này là ('khmt', 7, 9)
'''
}

normailize

In [6]:
df.dtb_hk = df.dtb_hk / 10

In [7]:
df.describe()

,dtb_hk
count,100804.000000
mean,0.650092
std,0.205820
min,0.000000
25%,0.570000
50%,0.700000
75%,0.793182
max,1.000000


construct dataset

In [8]:
group_df = df.sort_values(by=['nam_hoc', 'hoc_ky']).groupby(['mssv', 'khoa'])

In [9]:
sv_dtb_hk = group_df.apply(lambda x: (x.khoa.iloc[0], list(x.dtb_hk)))
sv_dtb_hk

mssv                                      khoa  
0000AC05XPvAibaEXe9B2tolTZ0JLoBGbkQixQS6  mmt&tt    (mmt&tt, [0.7807692289352417, 0.82749998569488...
0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G  httt      (httt, [0.889090895652771, 0.8912500143051147,...
00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD  ktmt      (ktmt, [0.5277777910232544, 0.7124999761581421...
0006A0BBXPvAibaEXe/lMOwHQdw54DgUkWaqwb1u  khmt                            (khmt, [0.721818208694458])
000AD0D8XPvAibaEXe+RQyZpP6sq6qqIPZXybx3Q  cnpm      (cnpm, [0.8858332633972168, 0.8619999885559082...
                                                                          ...                        
FFDFD671XPvAibaEXe8/3iNqySORb8gPuwsQ8fwa  httt      (httt, [0.6555555462837219, 0.4499999880790710...
FFE9E452XPvAibaEXe+6MN9FoW2mqXtx1lMTF9+D  mmt&tt    (mmt&tt, [0.8138889074325562, 0.80312502384185...
FFF3D630XPvAibaEXe8byJxgVsE7R2dP7ICEdFWK  httt      (httt, [0.90666663646698, 0.7772727608680725, ...
FFF4CD57XPvAibaEXe/z8kRiyNmKNla42

In [10]:
new_dataset['input'] = []
new_dataset['target'] = []
log_times = 3
count = 0
for khoa, dtb_hk in sv_dtb_hk:
    if (count < log_times):
        print('haha', dtb_hk)

    for i in range(1, len(dtb_hk) +1 - num_dtb):
        new_dataset['input'].append(tuple([khoa] + dtb_hk[:i]))
        new_dataset['target'].append(tuple(dtb_hk[i:i + num_dtb]))

        if (count < log_times):
            print(new_dataset['input'][-1], new_dataset['target'][-1])

    if (count < log_times):
        count += 1

haha [0.7807692289352417, 0.8274999856948853, 0.7508333325386047]
('mmt&tt', 0.7807692289352417) (0.8274999856948853,)
('mmt&tt', 0.7807692289352417, 0.8274999856948853) (0.7508333325386047,)
haha [0.889090895652771, 0.8912500143051147, 0.9053846597671509, 0.8758824467658997, 0.8535000681877136, 0.8616666793823242, 0.8411110639572144]
('httt', 0.889090895652771) (0.8912500143051147,)
('httt', 0.889090895652771, 0.8912500143051147) (0.9053846597671509,)
('httt', 0.889090895652771, 0.8912500143051147, 0.9053846597671509) (0.8758824467658997,)
('httt', 0.889090895652771, 0.8912500143051147, 0.9053846597671509, 0.8758824467658997) (0.8535000681877136,)
('httt', 0.889090895652771, 0.8912500143051147, 0.9053846597671509, 0.8758824467658997, 0.8535000681877136) (0.8616666793823242,)
('httt', 0.889090895652771, 0.8912500143051147, 0.9053846597671509, 0.8758824467658997, 0.8535000681877136, 0.8616666793823242) (0.8411110639572144,)
haha [0.5277777910232544, 0.7124999761581421, 0.790000021457672

In [11]:
len(new_dataset['input']), len(new_dataset['target'])

(86444, 86444)

In [12]:
print(new_dataset['mo_ta'])


==================     VERSION dataset_1-output_khoa-string     ==================
1. một mẩu của target là MỘTMỘTMỘT dtb_hk
2. một mẩu của input là list dtb_hk trong các học kỳ trước hk trong target
và điểm này thuộc sinh viên tương ứng target
3. diem được normalize từ [0-10] về [0-1]
4. input và target được sắp xếp theo thứ tự học kỳ tăng dần
5. input và target chưa được shuffle
6. Các học kỳ có giá trị null đã được loại bỏ. 
Xem như học kỳ trước và sau của học kỳ null đó là 2 học kỳ liên tiếp
7. HỌC KỲ 3 KHÔNGKHÔNGKHÔNG ĐƯỢC DROP
---
8. Input thêm khoa dạng string vào đầu tiên
    - Ví dụ input data_for_model v1, v2 là (7, 9)
    - Nếu sinh viên thuộc KHMT, input của dataset này là ('khmt', 7, 9)



In [13]:
with open(dataset_dir/f'{version_name}.pkl', 'wb') as file:
    pickle.dump(new_dataset, file)